# フレア検出 学習ノート 04: 恒星ごとの特殊処理（DS Tuc A / V889 Her）

このノートでは、`FlareDetector_DS_Tuc_A` と `FlareDetector_V889_Her` を題材に、
Base クラス `BaseFlareDetector` をどのように拡張しているかを学びます。

- DS Tuc A: 連星系＋トランジットを持つ若い星
- V889 Her: 強い活動性を持つ回転変動の大きな星

それぞれの恒星に合わせて「どの段階を」「どのように」カスタマイズしているかを、
コードと図を使って確認していきます。


## このノートで学ぶこと

- なぜ恒星ごとに `remove` / `detrend_flux` / `tess_band_energy` / `flux_diff` を
  カスタマイズする必要があるのか
- DS Tuc A のトランジット除去ロジックと連星補正の考え方
- V889 Her の「フレアだけを埋めてから」デトレンドする高度な手法
- Base クラスとサブクラスの責務分担の整理

前提として、01〜03 のノートで基本フローを一通り体験していることを想定しています。


In [1]:
import sys
from pathlib import Path

NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("PROJECT_ROOT:", PROJECT_ROOT)

NOTEBOOK_DIR: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/notebooks/learning
PROJECT_ROOT: /Users/daisukeyamashiki/Documents/kyoto-flare-detection


## 1. なぜ恒星ごとの実装が必要か

`BaseFlareDetector` は「単一星」を前提にした共通ロジックを提供しますが、
実際には恒星ごとに次のような違いがあります。

- 連星系かどうか（DS Tuc A は主星 + 伴星）
- トランジットや強い系外惑星信号があるかどうか
- 長期変動やフレアの典型的な形がどれくらい複雑か（V889 Her は強い活動星）

そのため、

- DS Tuc A では `remove()` と `tess_band_energy()` / `flux_diff()` をオーバーライドして
  連星補正とトランジット除去を行う
- V889 Her では `detrend_flux()` をオーバーライドして
  高度なフレアキャンセル付きデトレンドを行う

といった「星ごとの拡張」が入っています。

このノートでは、実際のクラスを使って

- DS Tuc A のトランジット除去と連星補正
- V889 Her の高度デトレンド

を、図と数値の両方から確認していきます。


In [2]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from src.flarepy_DS_Tuc_A import FlareDetector_DS_Tuc_A
from src.flarepy_V889_Her import FlareDetector_V889_Her

DATA_DIR_DS_TUC_A = PROJECT_ROOT / "data" / "TESS" / "DS_Tuc_A"
DATA_DIR_V889_HER = PROJECT_ROOT / "data" / "TESS" / "V889_Her"

print("DATA_DIR_DS_TUC_A:", DATA_DIR_DS_TUC_A)
print("DATA_DIR_V889_HER:", DATA_DIR_V889_HER)


DATA_DIR_DS_TUC_A: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/DS_Tuc_A
DATA_DIR_V889_HER: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/V889_Her


## 2. DS Tuc A: トランジット除去と連星補正

DS Tuc A は主星＋伴星からなる連星系で、かつトランジットを持つ系外惑星が知られています。
そのため、TESS の光度曲線には

- 想定外のディッピング（トランジット）
- 連星伴星による「余分な光」

が含まれています。そのままフレア検出やエネルギー計算を行うと、

- トランジットの谷をフレアと誤検出してしまう
- 伴星の光まで含めてエネルギーを見積もってしまう

といった問題が起きます。

この節では、

1. `remove()` によるトランジット区間のマスク
2. `tess_band_energy()` / `flux_diff()` による「主星＋伴星」を考慮したスケーリング

を、実際の光度曲線と数値で確認します。


In [3]:
if not DATA_DIR_DS_TUC_A.exists():
    print("DS Tuc A のディレクトリが見つかりません。data/TESS/DS_Tuc_A を確認してください。")
else:
    fits_files_ds = sorted(DATA_DIR_DS_TUC_A.glob("*.fits"))
    if not fits_files_ds:
        print("DS Tuc A の FITS ファイルが見つかりません。")
    else:
        ds_file = fits_files_ds[0]
        print("使用するファイル:", ds_file.name)

        # トランジット除去前の光度曲線
        ds_raw = FlareDetector_DS_Tuc_A(file=str(ds_file), process_data=False)
        time_raw = ds_raw.atessBJD
        flux_raw = ds_raw.amPDCSAPflux

        # remove() 適用後の光度曲線
        ds_removed = FlareDetector_DS_Tuc_A(file=str(ds_file), process_data=False)
        ds_removed.remove()
        time_removed = ds_removed.tessBJD
        flux_removed = ds_removed.mPDCSAPflux

        fig = make_subplots(
            rows=2,
            cols=1,
            shared_xaxes=True,
            vertical_spacing=0.05,
            subplot_titles=("トランジット除去前", "トランジット除去後"),
        )

        fig.add_trace(
            go.Scatter(x=time_raw, y=flux_raw, mode="lines", name="raw"),
            row=1,
            col=1,
        )
        fig.add_trace(
            go.Scatter(x=time_removed, y=flux_removed, mode="lines", name="removed", line=dict(color="orange")),
            row=2,
            col=1,
        )

        fig.update_xaxes(title_text="Time [BJD]", row=2, col=1)
        fig.update_yaxes(title_text="Normalized Flux", row=1, col=1)
        fig.update_yaxes(title_text="Normalized Flux", row=2, col=1)
        fig.update_layout(height=600)
        fig.show()


使用するファイル: tess2018206045859-s0001-0000000410214986-0120-s_lc.fits


### 図の読み方

- 上段: 元の正規化光度。トランジットによる深い谷が周期的に現れています。
- 下段: `remove()` 実行後。トランジットに対応する時間帯がマスクされ、
  連続した回転変動＋フレアの上に乗った曲線になります。

この処理により、フレア検出アルゴリズムは
「トランジットの谷」をフレアと誤認識しにくくなります。


In [4]:
if 'ds_file' not in globals():
    print("先に DS Tuc A のトランジット除去セルを実行してください。")
else:
    # process_data=True でフルパイプラインを実行
    ds_proc = FlareDetector_DS_Tuc_A(file=str(ds_file), process_data=True)

    print("--- DS Tuc A summary ---")
    print("flare_number:", ds_proc.flare_number)
    print("sum_flare_energy [erg]:", ds_proc.sum_flare_energy)
    print("precise_obs_time [day]:", ds_proc.precise_obs_time)
    if ds_proc.precise_obs_time > 0:
        print("flare_ratio [1/day]:", ds_proc.flare_number / ds_proc.precise_obs_time)
        print("energy_ratio [erg/day]:", ds_proc.sum_flare_energy / ds_proc.precise_obs_time)
    else:
        print("flare_ratio / energy_ratio は obs time が 0 のため計算できません。")
    print("starspot [m^2]:", ds_proc.starspot)
    print("starspot_ratio:", ds_proc.starspot_ratio)

    if ds_proc.energy is not None and len(ds_proc.energy) > 0:
        sample_energy = float(ds_proc.energy[0])
        print("sample flare energy [erg]:", sample_energy)
    else:
        print("エネルギーが計算されていません。フレア検出結果を確認してください。")



--- DS Tuc A summary ---
flare_number: 6
sum_flare_energy [erg]: 1.145365094916553e+35
precise_obs_time [day]: 26.524874270412965
flare_ratio [1/day]: 0.22620276872311773
energy_ratio [erg/day]: 4.3180792611490137e+33
starspot [m^2]: 3.385568350740028e+17
starspot_ratio: 0.0740920429992965
sample flare energy [erg]: 1.5408579468834685e+34


DS Tuc A では、`tess_band_energy()` 内で

- 主星（R = 0.87 R☉, T ≈ 5428 K）
- 伴星（R ≈ 0.864 R☉, T ≈ 4700 K）

の両方の寄与を足し合わせて TESS 帯での光度を評価しています。
これにより、同じフラックス増分 `count` でも、
「連星系としての実効的な放射面積」に応じたエネルギーが見積もられます。

また、`flux_diff()` でも主星＋伴星の面積を使ってスポット面積を評価しており、
単一星の近似よりも現実的なスポットサイズが得られます。


## 3. V889 Her: フレアキャンセル付き高度デトレンド

V889 Her は強い活動性を持つ恒星で、

- 回転に伴う大きな長期変動
- 頻繁に起こるフレア

が同時に存在します。そのため「単純なローパスフィルタ」だけでは

- フレア成分まで一緒に平滑化してしまう
- 基本的な回転変動が十分に再現できない

といった問題が起こります。

`FlareDetector_V889_Her.detrend_flux()` では、まず差分列を使って
フレア候補区間を推定し、その部分をいったん補間で埋めてから
スプライン＋ローパスでベースラインを推定しています。
この手順により、

- フレアのピークはなるべく壊さずに
- 回転変動だけをきれいに取り除いたデトレンド光度

を得ることができます。


In [5]:
if not DATA_DIR_V889_HER.exists():
    print("V889 Her のディレクトリが見つかりません。data/TESS/V889_Her を確認してください。")
else:
    fits_files_v889 = sorted(DATA_DIR_V889_HER.glob("*.fits"))
    if not fits_files_v889:
        print("V889 Her の FITS ファイルが見つかりません。")
    else:
        v_file = fits_files_v889[0]
        print("使用するファイル:", v_file.name)

        v_det = FlareDetector_V889_Her(file=str(v_file), process_data=True)

        time_ext = v_det.gtessBJD
        flux_ext = v_det.gmPDCSAPflux
        detrended = v_det.s2mPDCSAPflux

        fig = make_subplots(
            rows=2,
            cols=1,
            shared_xaxes=True,
            vertical_spacing=0.05,
            subplot_titles=("gap 補正後の光度曲線", "高度デトレンド後 (s2mPDCSAPflux)"),
        )

        fig.add_trace(
            go.Scatter(x=time_ext, y=flux_ext, mode="lines", name="gap corrected", line=dict(width=1)),
            row=1,
            col=1,
        )

        fig.add_trace(
            go.Scatter(x=v_det.tessBJD, y=detrended, mode="lines", name="detrended", line=dict(width=1, color="orange")),
            row=2,
            col=1,
        )

        fig.update_xaxes(title_text="Time [BJD]", row=2, col=1)
        fig.update_yaxes(title_text="Normalized Flux", row=1, col=1)
        fig.update_yaxes(title_text="Detrended Flux", row=2, col=1)
        fig.update_layout(height=600)
        fig.show()


使用するファイル: tess2020160202036-s0026-0000000471000657-0188-s_lc.fits


### difference_at_lag によるフレア候補抽出のイメージ

実装では、

- `diff_flux = np.diff(flux_ext)`
- `difference_at_lag(flux_ext, n=2..5)`

といった複数のラグ差分を取り、

- どれかの差分が 0.01 以上
- 時間差分 `diff_time` が極端に大きくない

という条件を満たす点を「フレアが始まり得る場所」として候補に挙げています。
その後、フレア前後で元の値に戻るまでの区間を探し、
その区間を補間で埋めてからスプライン・ローパスを適用することで、
長期変動の推定を安定させています。


## 4. Base クラスとサブクラスの責務分担の整理

ここまでの内容を踏まえて、設計上の役割分担を整理すると次のようになります。

- BaseFlareDetector
  - TESS LC の読み込み (`load_TESS_data`)
  - ギャップ補正 (`apply_gap_correction`)
  - 一般的なローパス＆スプラインによるデトレンド (`detrend_flux`)
  - 誤差の再推定 (`reestimate_errors`)
  - フレア検出・エネルギー・回転周期・スポット指標の計算
- FlareDetector_DS_Tuc_A
  - `remove()` でトランジット区間を明示的にマスク
  - `tess_band_energy()` / `flux_diff()` で主星＋伴星を考慮したスケーリング
- FlareDetector_V889_Her
  - `detrend_flux()` を上書きし、フレアを一時的に埋めてからデトレンド

このように「共通ロジックは Base クラス」「恒星ごとの差分はサブクラス」で分離しておくと、

- 新しい恒星を追加するときに、必要な部分だけをオーバーライドすればよい
- 既存のアルゴリズム改善を Base クラスに集約できる

といったメリットがあります。
